# Main Analysis

This file does not rely on user input. So we would like to keep it interchangeable with the actual main_analysis.py file.
So please keep all the >relevant< code within one code block. (If you want to inspect certain elements or do other things that are not relevant to the main analysis, you can of course add them in new blocks).

In [12]:
import sys
import os.path as op
module_path = op.abspath(op.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import mne

from data_analysis.functions_preprocessing import \
    (combine_raws, split_raws, preprocess_single_sub, mark_bads_and_save,
     run_ica_and_save)
from data_analysis.functions_behavioral import \
    (create_event_df, remove_ghost_triggers, calculate_alpha,
     join_event_dfs, remove_outliers, events_from_event_df)
from data_analysis.functions_connectivity import \
    full_ispc
from data_analysis.functions_graph_theory import \
    weighted_small_world_coeff



subject_dir = "/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/"
behav_dir = "/net/store/nbp/projects/hyperscanning/study_project/NBP_Hyperscanning/data_analysis/Behavioural_Analysis/BehaviouralData"



# Main Data Analysis ###################################
# initialize containers to analyze later
connectivity_matrices = []
small_world_coeffs = []

# Perform the data analysis
for subj_pair in ['202', '203']:  #['202','203','204','205','206','207','208','209','211','212']:

    subs_path = subject_dir + "sub-{0}/eeg/sub-{0}_task-hyper_eeg.fif".format(subj_pair)
    behav_path = op.join(behav_dir, "{0}.csv".format(subj_pair))

    combined_raw = mne.io.read_raw_fif(subs_path, preload=True)

    # split the subjects and delete the raw file
    raws = split_raws(combined_raw)
    del combined_raw

    # apply the preprocessing
    #for idx, raw in enumerate(raws):
    #    subj_idx = "sub-{0}_p-{1}".format(subj_pair, idx)
    #    raws[idx] = preprocess_single_sub(raw, subj_idx)


    # combine the subjects again
    raw_combined = combine_raws(raws[0], raws[1])
    del raws  # to save memory

    # do the behavioral analysis and get the epochs
    behavioral_df = calculate_alpha(pd.read_csv(behav_path))
    
    event_df = create_event_df(raw_combined)
    event_df = remove_ghost_triggers(event_df)
    event_df = join_event_dfs(event_df, behavioral_df)
    event_df = remove_outliers(event_df, exclude_stddev=2)
    events = events_from_event_df(event_df)

    # define the parameters for epoching
    # TODO: Define events more elaborate!
    event_id = 11
    tmin = 0
    tmax = 3

    # epoch the data. Here we filter out bad segments from both participants
    # TODO: do we need a baseline for the connectivity analysis?
    epochs = mne.Epochs(raw_combined, events, event_id, tmin, tmax,
                        picks=["eeg"], baseline=(0, 0),
                        reject_by_annotation=True)

    # Perform the high level analysis for each epoch
    for epoch in epochs[:1]:

        # calculate the ISPC
        ispc_matrix, freqs = full_ispc(epoch, epochs.info["sfreq"],
                                       nperseg=32)

        # calculate the small world coefficient for each frequency
        small_worlds = [weighted_small_world_coeff(ispc_matrix[:, :, i].squeeze())
                        for i in range(ispc_matrix.shape[2])]

        # append the results to the respective lists
        connectivity_matrices.append(ispc_matrix)
        small_world_coeffs.append(small_worlds)

Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg.fif...


<ipython-input-12-11ccd6f1dea6>:39: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  combined_raw = mne.io.read_raw_fif(subs_path, preload=True)


Isotrak not found
    Range : 0 ... 3725311 =      0.000 ...  3637.999 secs
Ready.
Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg-1.fif...
Isotrak not found
    Range : 3725312 ... 5675445 =   3638.000 ...  5542.427 secs
Ready.
Reading 0 ... 5675445  =      0.000 ...  5542.427 secs...
6648 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]
288 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


/net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/functions_connectivity.py:34: ComplexWarning: Casting complex values to real discards the imaginary part
  ispc_matrix[chan_a, chan_b, :] = np.array(


Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-203/eeg/sub-203_task-hyper_eeg.fif...


<ipython-input-12-11ccd6f1dea6>:39: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-203/eeg/sub-203_task-hyper_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  combined_raw = mne.io.read_raw_fif(subs_path, preload=True)


Isotrak not found
    Range : 0 ... 3725311 =      0.000 ...  3637.999 secs
Ready.
Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-203/eeg/sub-203_task-hyper_eeg-1.fif...
Isotrak not found
    Range : 3725312 ... 4566330 =   3638.000 ...  4459.307 secs
Ready.
Reading 0 ... 4566330  =      0.000 ...  4459.307 secs...
6649 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]
279 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


/net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/functions_connectivity.py:34: ComplexWarning: Casting complex values to real discards the imaginary part
  ispc_matrix[chan_a, chan_b, :] = np.array(


In [11]:
#event_df[event_df["alpha"] > 360]
event_df

,sample,event_code,event_index,event_name,pair,condition,block,trial,subject,tapnr,ttap,jitter,player_start_first,diff_inter,diff_intra,alpha,alpha_lin
0,773771,15,17,s2/t1,203,blind,1,1,1,1,3.488252,0.075,1,0.108922,0.660805,59.339627,59.339627
9,773882,6,18,s1/t1,203,blind,1,1,2,1,3.379330,0.075,1,-0.108922,0.714642,54.869318,54.869318
1,774503,16,19,s2/t2,203,blind,1,1,1,2,4.149057,0.075,1,0.055085,0.669061,29.639412,29.639412
10,774559,7,20,s1/t2,203,blind,1,1,2,2,4.093972,0.075,1,-0.055085,0.658083,30.133860,30.133860
2,775176,17,21,s2/t3,203,blind,1,1,1,3,4.818118,0.075,1,0.066063,0.655882,36.260705,36.260705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5397,4527577,12,6642,s1/t7,203,blind,12,300,2,7,6.136573,0.075,1,-0.038842,0.500590,27.933260,27.933260
5389,4528034,13,6643,s1/t8,203,blind,12,300,1,8,6.621934,0.075,1,-0.015229,0.943700,5.809510,5.809510
5398,4528050,22,6644,s2/t8,203,blind,12,300,2,8,6.637163,0.075,1,0.015229,0.467523,11.726555,11.726555
5390,4528528,23,6645,s2/t9,203,blind,12,300,1,9,7.565634,0.075,1,0.460948,NaN,NaN,NaN
